In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark

# install pyspark
!pip3 install pyspark==3.2.0

# install graphframes
!pip3 install graphframes

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=32be791c393da838612d511b653607cd3087b7bea1dc4051efe2afd305e27124
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 154 kB 5.1 MB/s 


In [ ]:
!cp -v /content/graphframes-0.8.2-spark3.2-s_2.12.jar $SPARK_HOME/jars/

'/content/graphframes-0.8.2-spark3.2-s_2.12.jar' -> '/content/spark-3.2.0-bin-hadoop3.2/jars/graphframes-0.8.2-spark3.2-s_2.12.jar'


In [ ]:
#import the packages
from pyspark import *
from pyspark.sql import *
from graphframes import *
import findspark
import pandas as pd

findspark.init()

# Start a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# access the postgresql server
conn = psycopg2.connect(
    host="codd04.research.northwestern.edu",
    port = "5433",
    database="postgres",
    user="cpdbstudent",
    password="DataSci4AI")

In [ ]:
cursor = conn.cursor()

In [ ]:
# Selecting for officers with GREATER than average number of award types 
edges_query = '''--Co-accusals of officers with DAM allegations earning less than average distinct award types
DROP TABLE IF EXISTS award_count;
CREATE TEMP TABLE award_count AS (
    SELECT officer_id offid, count(award_type) num_awards
    FROM data_award
    WHERE data_award.officer_id IN
                  (SELECT id
                  FROM data_officer
                  WHERE data_officer.id IN
                    (SELECT officer_id
                    FROM data_officerallegation
                    WHERE data_officerallegation.allegation_category_id IN
                      (SELECT id
                      FROM data_allegationcategory
                      WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
                      OR data_allegationcategory.category_code IN ('024', '003', '003A', '003B', '003C', '003D', '003E'))))
    group by officer_id);

DROP TABLE IF EXISTS da_category_ids;
CREATE TEMP TABLE da_category_ids AS (
    SELECT id
    FROM data_allegationcategory
    WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
    OR data_allegationcategory.category_code IN ('08J', '024', '003', '003A', '003B', '003C', '003D', '003E'));

DROP TABLE IF EXISTS da_cohort;
CREATE TEMP TABLE da_cohort AS (
    SELECT DISTINCT officer_id
    FROM data_officerallegation
    WHERE allegation_category_id IN (SELECT * from da_category_ids));

DELETE FROM da_cohort c
USING award_count a WHERE a.offid = c.officer_id
AND a.num_awards < (SELECT AVG(num_awards) FROM award_count);

DROP TABLE IF EXISTS class_example;
CREATE TEMP TABLE class_example AS (
SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship
FROM data_officerallegation da1
JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id
GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC);

DROP TABLE IF EXISTS edgesq2;
CREATE TEMP TABLE edgesq2 AS (
SELECT src,dst,relationship FROM class_example
JOIN da_cohort ON da_cohort.officer_id = class_example.src);

select * from edgesq2;
'''

In [ ]:
# Selecting for officers with GREATER than average number of award types 
nodes_query = '''--Co-accusals of officers with DAM allegations earning less than average distinct award types
DROP TABLE IF EXISTS award_count;
CREATE TEMP TABLE award_count AS (
    SELECT officer_id offid, count(award_type) num_awards
    FROM data_award
    WHERE data_award.officer_id IN
                  (SELECT id
                  FROM data_officer
                  WHERE data_officer.id IN
                    (SELECT officer_id
                    FROM data_officerallegation
                    WHERE data_officerallegation.allegation_category_id IN
                      (SELECT id
                      FROM data_allegationcategory
                      WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
                      OR data_allegationcategory.category_code IN ('024', '003', '003A', '003B', '003C', '003D', '003E'))))
    group by officer_id);

DROP TABLE IF EXISTS da_category_ids;
CREATE TEMP TABLE da_category_ids AS (
    SELECT id
    FROM data_allegationcategory
    WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
    OR data_allegationcategory.category_code IN ('08J', '024', '003', '003A', '003B', '003C', '003D', '003E'));

DROP TABLE IF EXISTS da_cohort;
CREATE TEMP TABLE da_cohort AS (
    SELECT DISTINCT officer_id
    FROM data_officerallegation
    WHERE allegation_category_id IN (SELECT * from da_category_ids));

DELETE FROM da_cohort c
USING award_count a WHERE a.offid = c.officer_id
AND a.num_awards < (SELECT AVG(num_awards) FROM award_count);

DROP TABLE IF EXISTS class_example;
CREATE TEMP TABLE class_example AS (
SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship
FROM data_officerallegation da1
JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id
GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC);

DROP TABLE IF EXISTS edgesq2;
CREATE TEMP TABLE edgesq2 AS (
SELECT src,dst,relationship FROM class_example
JOIN da_cohort ON da_cohort.officer_id = class_example.src);

select * from edgesq2;

DROP TABLE IF EXISTS withoutRelation;
CREATE TEMP TABLE withoutRelation AS (
select id,first_name || ' ' || last_name officer_name from data_officer where id in
(select src from edgesq2) and id in (select dst from edgesq2));

DROP TABLE IF EXISTS award_count;
CREATE TEMP TABLE award_count AS (
    SELECT officer_id offid, count(award_type) num_awards
    FROM data_award
    WHERE data_award.officer_id IN
                  (SELECT id
                  FROM data_officer
                  WHERE data_officer.id IN
                    (SELECT officer_id
                    FROM data_officerallegation
                    WHERE data_officerallegation.allegation_category_id IN
                      (SELECT id
                      FROM data_allegationcategory
                      WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
                      OR data_allegationcategory.category_code IN ('024', '003', '003A', '003B', '003C', '003D', '003E'))))
    group by officer_id);


select withoutRelation.id,withoutRelation.officer_name,award_count.num_awards from withoutRelation
join award_count on withoutRelation.id=award_count.offid;


'''

In [ ]:
import pandas as pd

In [ ]:
cursor.execute(edges_query)
edges = cursor.fetchall()
print("shape is: " + str(len(edges))) # 17465

df_edges = pd.DataFrame(edges)
colnames = [desc[0] for desc in cursor.description]
df_edges.columns = colnames

print(df_edges)

shape is: 14338
         src    dst  relationship
0          9  12641             1
1         54  27802             1
2         54  29189             1
3         54  30398             1
4         54  31550             1
...      ...    ...           ...
14333  32346  32361             3
14334  32346  32351             2
14335  32346  32377             3
14336  32397  32419             1
14337  32397  32406             1

[14338 rows x 3 columns]


In [ ]:
cursor.execute(nodes_query)
nodes = cursor.fetchall()
print("shape is: " + str(len(nodes))) # 17465

df_nodes = pd.DataFrame(nodes)
colnames = [desc[0] for desc in cursor.description]
df_nodes.columns = colnames

print(df_nodes)

shape is: 127
        id       officer_name  num_awards
0     2397      Rudolph Booth          27
1     3965  Anthony Carothers          26
2     4102       Frank Casale          33
3     4987    Garland Coleman          90
4     5094        Dean Coltri          66
..     ...                ...         ...
122  32069       Andrew Janik         177
123  32091        Skip Katich          73
124  32178       Brian Mchale          72
125  32181       Paul Meagher          49
126  32346         Mark Smith          59

[127 rows x 3 columns]


In [ ]:
edges_ = spark.createDataFrame(df_edges)

In [ ]:
nodes = spark.createDataFrame(df_nodes)

In [ ]:
cpdb = GraphFrame(nodes, edges_)

In [ ]:
cpdb.vertices.show()

+----+-----------------+----------+
|  id|     officer_name|num_awards|
+----+-----------------+----------+
|2397|    Rudolph Booth|        27|
|3965|Anthony Carothers|        26|
|4102|     Frank Casale|        33|
|4987|  Garland Coleman|        90|
|5094|      Dean Coltri|        66|
|5158|    John Conneely|        41|
|5205|    Kevin Connors|        33|
|5293|     Carlton Cook|        22|
|5568|         Carl Cox|        22|
|5893|  Robert Cummings|       106|
|6088|      Sean Dailey|        61|
|6313|       Eric Davis|        23|
|6523|    Michael Deane|        26|
|7160|  William Donovan|        22|
|7698|      Barry Earls|       180|
|8245|      James Farah|        22|
|8457|     Vincent Fico|        63|
|8762| Theodore Floodas|        50|
|8799|    Samuel Flores|        26|
|8887|    John Folliard|        77|
+----+-----------------+----------+
only showing top 20 rows



In [ ]:
cpdb.edges.show()

+---+-----+------------+
|src|  dst|relationship|
+---+-----+------------+
|  9|12641|           1|
| 54|27802|           1|
| 54|29189|           1|
| 54|30398|           1|
| 54|31550|           1|
| 54|31756|           1|
| 54| 4168|           1|
| 54|11252|           1|
| 54|12499|           1|
| 54|14979|           1|
| 54|21441|           1|
| 54|23448|           1|
| 54|23722|           1|
| 54|23778|           1|
| 54|24564|           1|
| 54|26802|           1|
| 54|27250|           1|
| 54|27088|           1|
| 54|10945|           2|
| 54| 1984|           2|
+---+-----+------------+
only showing top 20 rows



In [ ]:
cpdb.vertices.sort(['id'],ascending=True).show()

+----+-----------------+----------+
|  id|     officer_name|num_awards|
+----+-----------------+----------+
|2397|    Rudolph Booth|        27|
|3965|Anthony Carothers|        26|
|4102|     Frank Casale|        33|
|4987|  Garland Coleman|        90|
|5094|      Dean Coltri|        66|
|5158|    John Conneely|        41|
|5205|    Kevin Connors|        33|
|5293|     Carlton Cook|        22|
|5568|         Carl Cox|        22|
|5893|  Robert Cummings|       106|
|6088|      Sean Dailey|        61|
|6313|       Eric Davis|        23|
|6523|    Michael Deane|        26|
|7160|  William Donovan|        22|
|7698|      Barry Earls|       180|
|8245|      James Farah|        22|
|8457|     Vincent Fico|        63|
|8762| Theodore Floodas|        50|
|8799|    Samuel Flores|        26|
|8887|    John Folliard|        77|
+----+-----------------+----------+
only showing top 20 rows



In [ ]:
pr_cpdb = cpdb.pageRank(resetProbability=0.15, tol=0.01)
#look at the pagerank score for every vertex
pr_cpdb.vertices.orderBy('pagerank', ascending=False).show()
pr_cpdb.edges.show()

+-----+----------------+----------+------------------+
|   id|    officer_name|num_awards|          pagerank|
+-----+----------------+----------+------------------+
|18958|  Diane Michaels|        38|1.6717113250776576|
|28878|   Joseph Treacy|       129|1.2591088504897885|
|31945|    Melvin Ector|        45|1.1892831327106976|
|29574|    William Vick|        31|1.1668348957616044|
|28275|Frederick Taylor|        78|1.1596556038826997|
|32181|    Paul Meagher|        49|1.1363736734977217|
|25299|  Daniel Sanchez|        25|1.1340528178229516|
|32346|      Mark Smith|        59|1.1320279790416539|
|32091|     Skip Katich|        73|1.1137584535741631|
|24918|    Gilbert Ruiz|       136|1.1097057212006216|
|26150|   Brian Sheehan|        29| 1.106981601850434|
|19867|   Kevin Mulcahy|        27| 1.097669911316994|
|31191|     Jimmy Woods|       145|1.0968520402734792|
|29150|   Akil Upchurch|        73|  1.09564863873333|
|22466|  Martin Philbin|        57|  1.09564863873333|
|15695|Wil